In [1]:
# %pip install torch torchvision transformers opencv-python pandas ultralytics moviepy ollama pytubefix
#%pip show moviepy

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pytubefix import YouTube
from pytubefix.cli import on_progress
import pandas as pd
from moviepy.video.io.VideoFileClip import VideoFileClip
from PIL import Image
import io
from tqdm import tqdm
import os
import ollama

In [3]:
df = pd.read_csv('Video_Details.csv', encoding='ISO-8859-1')

In [4]:
df.head(2)

,channel_name,video_id,video_title,published_datetime,duration,view_count,like_count,dislike_count,comment_count,description,thumbnail_url
0,@Mrwhosetheboss,neIYdLysqlk,I tested the Craziest Xiaomi Gadgets!,2024-11-13T12:04:54Z,PT27M53S,6679521,219704,0,8890,I'm still genuinely in shock how great some of...,https://i.ytimg.com/vi/neIYdLysqlk/hqdefault.jpg
1,@Mrwhosetheboss,YX8ks42Azn8,The TRIPLE FOLDING phone has a Problem.,2024-10-26T14:06:50Z,PT12M54S,3464013,111736,0,5932,This phone has a LOT of good....and a LOT of b...,https://i.ytimg.com/vi/YX8ks42Azn8/hqdefault.jpg


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   channel_name        45 non-null     object
 1   video_id            45 non-null     object
 2   video_title         45 non-null     object
 3   published_datetime  45 non-null     object
 4   duration            45 non-null     object
 5   view_count          45 non-null     int64 
 6   like_count          45 non-null     int64 
 7   dislike_count       45 non-null     int64 
 8   comment_count       45 non-null     int64 
 9   description         45 non-null     object
 10  thumbnail_url       45 non-null     object
dtypes: int64(4), object(7)
memory usage: 4.0+ KB


In [6]:
df['video_url'] = df['video_id'].apply(lambda x: f"https://www.youtube.com/watch?v={x}")

In [7]:
df.head(1)

,channel_name,video_id,video_title,published_datetime,duration,view_count,like_count,dislike_count,comment_count,description,thumbnail_url,video_url
0,@Mrwhosetheboss,neIYdLysqlk,I tested the Craziest Xiaomi Gadgets!,2024-11-13T12:04:54Z,PT27M53S,6679521,219704,0,8890,I'm still genuinely in shock how great some of...,https://i.ytimg.com/vi/neIYdLysqlk/hqdefault.jpg,https://www.youtube.com/watch?v=neIYdLysqlk


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   channel_name        45 non-null     object
 1   video_id            45 non-null     object
 2   video_title         45 non-null     object
 3   published_datetime  45 non-null     object
 4   duration            45 non-null     object
 5   view_count          45 non-null     int64 
 6   like_count          45 non-null     int64 
 7   dislike_count       45 non-null     int64 
 8   comment_count       45 non-null     int64 
 9   description         45 non-null     object
 10  thumbnail_url       45 non-null     object
 11  video_url           45 non-null     object
dtypes: int64(4), object(8)
memory usage: 4.3+ KB


In [9]:
test_df = df.head(1)

# Unified Code

In [10]:
# Helper function to extract frames from a video
def extract_frames(video_path, frame_rate=1):
    clip = VideoFileClip(video_path)
    fps = clip.fps
    frame_list = []
    timestamps = []

    for i, frame in enumerate(clip.iter_frames()):
        if i % int(fps / frame_rate) == 0:
            image = Image.fromarray(frame)
            buffer = io.BytesIO()
            image.save(buffer, format="JPEG")
            buffer.seek(0)
            frame_list.append(buffer)
            timestamps.append(i / fps)

    return frame_list, timestamps

# Object detection using LLaMA
def detect_objects(input_frames):
    message_list = []
    for frame in tqdm(input_frames, desc="Detecting Objects in Frames"):
        try:
            # Read the BytesIO object into raw bytes
            frame_bytes = frame.getvalue()
            
            # Use Ollama to send the frame for object detection
            res = ollama.chat(
                model="llava",
                messages=[
                    {
                        "role": "user",
                        "content": "Describe all objects seen in this image.",
                        "images": [frame_bytes],  # Pass raw bytes
                    }
                ],
            )
            message_list.append(res['message']['content'])
        except Exception as e:
            print(f"Failed to process frame: {e}")
            message_list.append("Error processing frame")
    return message_list

# Main function to analyze a single video
def analyze_video(video_path):
    print(f"Analyzing video: {video_path}...")
    frames, timestamps = extract_frames(video_path, frame_rate=0.02)
    print("Running object detection...")
    object_data = detect_objects(frames)

    # Combine data into a DataFrame
    data = {
        "Timestamp": timestamps,
        "Detected Objects": object_data,
    }
    df = pd.DataFrame(data)
    return df

# Function to download and analyze all videos in the DataFrame
def download_and_analyze_videos(df):
    os.makedirs("videos_downloaded", exist_ok=True)

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Downloading Videos"):
        try:
            url = row['video_url']
            video_id = row['video_id']
            download_path = "videos_downloaded"
            output_filename = f"{video_id}.mp4"

            # Download the video
            yt = YouTube(url, on_progress_callback=on_progress)
            ys = yt.streams.get_highest_resolution()
            ys.download(output_path=download_path, filename=output_filename)

            # Analyze the downloaded video
            video_path = os.path.join(download_path, output_filename)
            analysis_results = analyze_video(video_path)

            # Save the analysis results
            output_csv = os.path.join("videos_downloaded", f"{video_id}_analysis.csv")
            analysis_results.to_csv(output_csv, index=False)
            print(f"Analysis saved for {video_id} at {output_csv}")
        
        except Exception as e:
            print(f"Failed to process video {row['video_url']}: {e}")


In [11]:
# Run the workflow
download_and_analyze_videos(test_df)

Analyzing video: videos_downloaded/neIYdLysqlk.mp4....0%


/Users/Nicolas/.virtualenvs/r-tensorflow/lib/python3.12/site-packages/moviepy/video/io/ffmpeg_reader.py:157: UserWarning: In file videos_downloaded/neIYdLysqlk.mp4, 614400 bytes wanted but 0 bytes read at frame index 50189 (out of a total 50190 frames), at time 1672.97/1673.02 sec. Using the last valid frame instead.
  warnings.warn(


Running object detection...


Detecting Objects in Frames:  53%|█████▎    | 18/34 [13:14:27<11:46:11, 2648.19s/it]


KeyboardInterrupt: 